In [1]:
from __future__ import division
import pandas as pd
import glob
import os
from datetime import datetime
import pickle
from PIL import Image
from VAMPIRE_backend import cntarea,cntAR,cntExtent,cntEquiDia,cntsol,bdreg,pca_bdreg,clusterSM
from time import time
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
src = r'\\fatherserverdw\Bart\Stiffness\Morphology\3_31_22_Jude_Confocal_0.5kPa'
setpaths = [x[0] for x in os.walk(src)]
setpaths = [_ for _ in setpaths if 'Mask' in _]
setpaths

['\\\\fatherserverdw\\Bart\\Stiffness\\Morphology\\3_31_22_Jude_Confocal_0.5kPa\\DMSO HMGB1\\Mask',
 '\\\\fatherserverdw\\Bart\\Stiffness\\Morphology\\3_31_22_Jude_Confocal_0.5kPa\\DMSO P16\\Mask',
 '\\\\fatherserverdw\\Bart\\Stiffness\\Morphology\\3_31_22_Jude_Confocal_0.5kPa\\HMGB1\\Mask',
 '\\\\fatherserverdw\\Bart\\Stiffness\\Morphology\\3_31_22_Jude_Confocal_0.5kPa\\P16\\Mask']

In [3]:
N=50 #equidistant points along cell boundary
clnum = 10
modelname='Bart_ABCDE'
outpth = r'\\fatherserverdw\Bart\Stiffness\Morphology\3_31_22_Jude_Confocal_0.5kPa\vampoutput'

In [4]:
modelpth = r'{}\{}.pickle'.format(outpth,modelname)
setIDs = [os.path.basename(os.path.dirname(_)) for _ in setpaths] #p
conditions = [os.path.basename(os.path.dirname(os.path.dirname(_))) for _ in setpaths] #p
conditions,setIDs

(['3_31_22_Jude_Confocal_0.5kPa',
  '3_31_22_Jude_Confocal_0.5kPa',
  '3_31_22_Jude_Confocal_0.5kPa',
  '3_31_22_Jude_Confocal_0.5kPa'],
 ['DMSO HMGB1', 'DMSO P16', 'HMGB1', 'P16'])

In [ ]:
boundaries =[]
imageIDs=[]
for setpath,condition,setID  in zip(setpaths,conditions,setIDs):
    masks = glob.glob(os.path.join(setpath,'*.tiff'))
    imageID = [os.path.splitext(os.path.basename(_))[0] for _ in masks]
    imstack = [np.array(Image.open(im)) for im in masks]
    for idx,im in zip(imageID,imstack):
        labels = list(set(im.flatten()))[1:]
        # iterate through labeled object in image
        for objidx, lab in enumerate(labels):
            mask = np.array((im == lab).astype(int), dtype='uint8')
            contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            boundary = np.flip(np.squeeze(contours))
            boundaries.append(boundary)
            imageIDs.append(idx)

In [ ]:
df = pd.DataFrame({'boundary':boundaries,
                   'condition':[condition]*len(boundaries),
                   'setID':[setID]*len(boundaries),
                   'imageID':imageIDs
                   })
df

In [ ]:
df = df[df['boundary'].map(len) > 5]
df['Area'] = df['boundary'].apply(lambda row: cntarea(row))
df['AR'] = df['boundary'].apply(lambda row: cntAR(row))
df['Sol'] = df['boundary'].apply(lambda row: cntsol(row))
df['Extent'] = df['boundary'].apply(lambda row: cntExtent(row))
df['EquiDia'] = df['boundary'].apply(lambda row: cntEquiDia(row))
df.to_pickle(os.path.join(setpath,'boundary.pkl'))

In [55]:
print('boundary saved to : ',os.path.join(setpath,'boundary.pkl'))

boundary saved to :  \\fatherserverdw\Bart\Stiffness\Morphology\3_31_22_Jude_Confocal_0.5kPa\DMSO HMGB1\Mask\boundary.pkl


In [ ]:
start = time()
pkls = []
for setpath in setpaths:
    start = time()
    if not os.path.exists(os.path.join(setpath,'boundary.pkl')):continue
    pkl = pd.read_pickle(os.path.join(setpath,'boundary.pkl'))
    pkls.append(pkl)
pkls = pd.concat(pkls,ignore_index=True)
print(round(time()-start),'seconds')
pkls

In [ ]:
B=pkls['boundary']
VamModel = {
                "N": [],
                "bdrn": [],
                "mdd": [],
                "pc": [],
                "clnum": [],
                "pcnum": [],
                "mincms": [],
                "testmean": [],
                "teststd": [],
                "boxcoxlambda": [],
                "C": [],
                "Z": []
            }

build_model = True
bdpc, VamModel = bdreg(B, N, VamModel, build_model)
score, VamModel = pca_bdreg(bdpc, VamModel, True)
pcnum = None # none is 20 by default
IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, VamModel, True, None, None,modelname)
f = open(os.path.join(*[outpth, modelname, modelname+'.pickle']), 'wb')
pickle.dump(VamModel, f)
f.close()

In [ ]:
experimental = False

In [ ]:
try:
    f = open(modelpth, 'rb')
except:
    print('the model does not exist. please replace model name to the one you built')
vampire_model = pickle.load(f)
N = vampire_model['N']
clnum = vampire_model['clnum']
pcnum = vampire_model['pcnum']

In [ ]:
#apply model
for setidx, setpath in enumerate(setpaths):
    try:
        print(setidx,setpath)
        pkl = pd.read_pickle(os.path.join(setpath,'boundary.pkl'))
        B = pkl['contour']
        pkl = pkl.drop(['bbox','centroid','contour','type'], axis=1)
        bdpc, vampire_model = bdreg(B, N, vampire_model, build_model)
        score, vampire_model = pca_bdreg(bdpc, vampire_model, build_model)
        IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, vampire_model, False, condition[setidx], setID[setidx],modelname)

        datasheet = 'VAMPIRE datasheet ' + tag + '.csv'
        goodness = goodness.transpose()
        pkl['Shape mode']= pd.Series(IDX)
        pkl['Distance from cluster center'] = pd.Series(IDX_dist)
        for idx,column in enumerate(goodness):
            pkl['probability of shape mode '+str(idx)] = column
        pkl.to_csv(os.path.join(setpath, datasheet), index=False)
    except:
        print('error')
        continue

In [ ]:
# update_csv(IDX, IDX_dist, tag[setidx], setpath, goodness=goodness)